In [1]:
import sqlite3
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

# Load data

In [2]:
conn = sqlite3.connect('yelp_dataset_business.db') 
df_business= pd.read_sql("""SELECT * FROM business""",conn )
conn.close()

In [3]:
df_business.is_open = df_business.is_open.astype(int)
df_business["review_count"]= df_business["review_count"].astype(int)
df_business['attributes']= df_business['attributes'].apply(lambda x: eval(x) if x else None)
df_business['hours']= df_business['hours'].apply(lambda x: eval(x) if x else None)

In [4]:
attr = [col for col in df_business.attributes.explode().unique() if col is not None]
lst_of_attr_dict = []
for attr_dict in df_business.attributes:
    if not attr_dict:
        lst_of_attr_dict.append({})
        continue

    if 'BusinessParking' in attr_dict:
        if type(attr_dict['BusinessParking']) == str:
            attr_dict['BusinessParking'] = ('True' in attr_dict['BusinessParking'])

    lst_of_attr_dict.append(attr_dict)
attr_df = pd.DataFrame(lst_of_attr_dict, columns=attr)

for col in attr_df:
        attr_df[col] = attr_df[col].fillna(False).astype(bool)
    
df_business = pd.concat([df_business.reset_index().drop('index', axis=1), attr_df], axis=1)
df_business.drop(['attributes'], axis=1, inplace=True)
lst_of_time = []
for time_dict in df_business.hours:
    if not time_dict:
        lst_of_time.append({})
        continue
    lst_of_time.append(time_dict)
time_df = pd.DataFrame(lst_of_time)
df_business = pd.concat([df_business, time_df], axis=1).drop('hours', axis=1)